<a href="https://colab.research.google.com/github/JackGO-1998/XLE-energy-index-replication/blob/main/XLE_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#Load Packages

# Note: Install yfinance and tensorflow APIs only if you don't have them (e.g., you get an error message when referring to them in your code)

# !pip3 install yfinance
# !pip3 install tensorflow

import yfinance as yf
import os
import time
import numpy as np
import scipy as sci
import scipy.io as sio
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import StandardScaler
import tensorflow as tf
#from tensorflow.keras.layers import Dense
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


from numpy.random import seed
seed(100)
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [21]:
raw_XLE = pd.read_excel("https://github.com/JackGO-1998/XLE-energy-index-replication/raw/refs/heads/main/XLE%20holdings_6.2025.xlsx")
# get raw data on IBBQ, an ETF that tracks the NBI
print("Size of the dataset (row, col): ", raw_XLE.shape)
raw_XLE

Size of the dataset (row, col):  (26, 8)


,Name,Ticker,Identifier,SEDOL,Weight,Sector,Shares Held,Local Currency
0,EXXON MOBIL CORP,XOM,30231G102,2326618,23.304046,-,5.980112e+07,USD
1,CHEVRON CORP,CVX,166764100,2838555,15.059282,-,2.925024e+07,USD
2,CONOCOPHILLIPS,COP,20825C104,2685717,7.508996,-,2.232961e+07,USD
3,WILLIAMS COS INC,WMB,969457100,2967181,4.798737,-,2.267931e+07,USD
4,EOG RESOURCES INC,EOG,26875P101,2318024,4.400290,-,1.027480e+07,USD
5,KINDER MORGAN INC,KMI,49456B101,B3NQ4P8,3.825070,-,3.893952e+07,USD
6,MARATHON PETROLEUM CORP,MPC,56585A102,B3K3L40,3.731477,-,6.365258e+06,USD
7,ONEOK INC,OKE,682680103,2130109,3.668340,-,1.249770e+07,USD
8,PHILLIPS 66,PSX,718546104,B78C4Y8,3.590175,-,8.319001e+06,USD
9,SCHLUMBERGER LTD,SLB,806857108,2779201,3.570585,-,2.821805e+07,USD


In [24]:
XLE_tickers = raw_XLE['Ticker']
XLE_tickers
XLE_tickers.dropna()

,Ticker
0,XOM
1,CVX
2,COP
3,WMB
4,EOG
5,KMI
6,MPC
7,OKE
8,PSX
9,SLB


In [30]:
XLE_tickers= XLE_tickers.drop([22, 24])

KeyError: '[22, 24] not found in axis'

In [32]:
XLE_tickers.reset_index(drop=True)

,Ticker
0,XOM
1,CVX
2,COP
3,WMB
4,EOG
5,KMI
6,MPC
7,OKE
8,PSX
9,SLB


In [37]:
tickers = XLE_tickers.tolist()
df_data_download = yf.download(tickers,start='2020-06-12',end='2025-06-09', interval="1d", auto_adjust=True)

[*********************100%***********************]  23 of 24 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IXPM5']: HTTPError('HTTP Error 404: ')
[*********************100%***********************]  23 of 24 completed

In [41]:
df_data_download

Price      Adj Close      Close                                               \
Ticker         IXPM5        APA        BKR        COP       CTRA         CVX   
Date                                                                           
2020-06-12       NaN  12.306574  13.856308  36.345745  15.594074   74.200310   
2020-06-15       NaN  12.583625  14.212713  36.029903  15.073491   73.268692   
2020-06-16       NaN  13.021549  14.638658  37.492722  15.003561   75.517426   
2020-06-17       NaN  12.413818  13.995392  36.586777  14.677230   73.517670   
2020-06-18       NaN  12.708747  14.004086  36.944180  14.583991   73.838905   
...              ...        ...        ...        ...        ...         ...   
2025-06-02       NaN  17.330000  37.320000  86.720001  25.080000  137.839996   
2025-06-03       NaN  18.270000  37.540001  87.910004  25.440001  139.550003   
2025-06-04       NaN  17.920000  37.290001  85.949997  24.889999  137.369995   
2025-06-05       NaN  17.850000  37.150002  85.349998  24.830000  136.899994   
2025-06-06       NaN  18.500000  37.900002  87.220001  25.580000  140.210007   

Price                                                     ...   Volume  \
Ticker            DVN         EOG        EQT         EXE  ...      MPC   
Date                                                      ...            
2020-06-12   9.613274   41.296558  13.995530         NaN  ...  7688900   
2020-06-15   9.680867   41.013760  13.598627         NaN  ...  9565800   
2020-06-16   9.988792   42.891220  13.305676         NaN  ...  7253800   
2020-06-17   9.485597   41.225849  13.248975         NaN  ...  5498500   
2020-06-18   9.725927   41.579357  12.927673         NaN  ...  6219300   
...               ...         ...        ...         ...  ...      ...   
2025-06-02  30.922300  111.339996  56.660000  119.300003  ...  2960500   
2025-06-03  31.786217  112.820000  56.650002  118.220001  ...  2004700   
2025-06-04  31.339365  110.769997  55.490002  116.199997  ...  2582200   
2025-06-05  31.200344  111.029999  55.330002  115.570000  ...  1532900   
2025-06-06  32.064262  114.050003  56.389999  117.349998  ...  1351700   

Price                                                                        \
Ticker           OKE       OXY      PSX       SLB     TPL     TRGP      VLO   
Date                                                                          
2020-06-12  31498500  44937900  3025700  14646700  147900  5819100  3575200   
2020-06-15  12268600  46134000  4308900  12742000  147900  3841500  4972900   
2020-06-16   9837200  57505000  3419100  17831200   83400  3932500  4046100   
2020-06-17   5267100  38371600  3665500  15816600   31200  2549800  4649800   
2020-06-18   4244400  36051700  3760200  15867700   98100  2302100  3817800   
...              ...       ...      ...       ...     ...      ...      ...   
2025-06-02   4111500   8003700  2841300  12681500  156700  1815700  2824500   
2025-06-03   3902000  12204400  2065500  16171900  152300  1979800  2310700   
2025-06-04   6153200  11577600  3686100  13545100  173500  2487500  4458400   
2025-06-05   5371600   7011700  2836300  11545800  137300  1903500  1845400   
2025-06-06   3617100   6850100  2821100  10410100  109200  1921500  2607400   

Price                           
Ticker           WMB       XOM  
Date                            
2020-06-12   9381300  30978100  
2020-06-15  13572900  27929300  
2020-06-16   9298500  26203400  
2020-06-17   6779700  19279400  
2020-06-18   8987900  21478700  
...              ...       ...  
2025-06-02  11558300  17290500  
2025-06-03   9977400  19194300  
2025-06-04   7445100  15648500  
2025-06-05   4844600  13924800  
2025-06-06   5273300  15487800  

[1253 rows x 121 columns]

In [44]:
holdingsreturn = df_data_download['Close']
holdingsreturn.drop('IXPM5',axis=1)

Ticker,APA,BKR,COP,CTRA,CVX,DVN,EOG,EQT,EXE,FANG,...,MPC,OKE,OXY,PSX,SLB,TPL,TRGP,VLO,WMB,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-12,12.306574,13.856308,36.345745,15.594074,74.200310,9.613274,41.296558,13.995530,NaN,37.787609,...,31.526989,24.575340,17.601261,62.830383,17.122240,194.334686,17.786146,52.063042,13.960499,37.585426
2020-06-15,12.583625,14.212713,36.029903,15.073491,73.268692,9.680867,41.013760,13.598627,NaN,38.721035,...,32.577309,26.269445,18.124550,61.480412,17.410618,193.891449,19.897287,51.899807,14.642059,37.561527
2020-06-16,13.021549,14.638658,37.492722,15.003561,75.517426,9.988792,42.891220,13.305676,NaN,40.209686,...,33.266048,27.682405,19.304306,62.651440,17.617882,198.782379,20.685572,53.997353,14.825849,38.406151
2020-06-17,12.413818,13.995392,36.586777,14.677230,73.517670,9.485597,41.225849,13.248975,NaN,37.457684,...,31.966059,26.074804,18.115036,59.268414,17.870214,193.751953,19.244915,50.863285,14.534848,37.155159
2020-06-18,12.708747,14.004086,36.944180,14.583991,73.838905,9.725927,41.579357,12.927673,NaN,37.489880,...,33.119698,26.204569,18.714426,62.000874,18.185621,190.584366,19.679836,52.748615,14.894778,37.386227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-02,17.330000,37.320000,86.720001,25.080000,137.839996,30.922300,111.339996,56.660000,119.300003,137.589996,...,158.750000,82.510002,41.209988,114.279999,33.061436,1110.140015,162.350006,128.300003,60.760098,103.050003
2025-06-03,18.270000,37.540001,87.910004,25.440001,139.550003,31.786217,112.820000,56.650002,118.220001,142.279999,...,161.970001,83.239998,42.453049,115.970001,33.825001,1126.369995,164.929993,130.949997,60.601429,103.800003
2025-06-04,17.920000,37.290001,85.949997,24.889999,137.369995,31.339365,110.769997,55.490002,116.199997,139.350006,...,156.949997,80.519997,41.677380,112.480003,33.240002,1102.530029,160.679993,126.540001,59.619667,102.300003


In [45]:
# Compute the log-returns and replace the columns entries, which previously were closing prices,
# with the log-returns (and relabel the columns as well)
for i in range(len(holdingsreturn.columns)):
    holdingsreturn.loc[:,holdingsreturn.columns[i]]=np.log(holdingsreturn.loc[:,holdingsreturn.columns[i]]/holdingsreturn.loc[:,holdingsreturn.columns[i]].shift(1))
    holdingsreturn.rename(columns={holdingsreturn.columns[i]:holdingsreturn.columns[i]+' log retn'}, inplace=True)

<ipython-input-45-459544639>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holdingsreturn.rename(columns={holdingsreturn.columns[i]:holdingsreturn.columns[i]+' log retn'}, inplace=True)


In [46]:
holdingsreturn

Ticker,APA log retn,BKR log retn,COP log retn,CTRA log retn,CVX log retn,DVN log retn,EOG log retn,EQT log retn,EXE log retn,FANG log retn,...,MPC log retn,OKE log retn,OXY log retn,PSX log retn,SLB log retn,TPL log retn,TRGP log retn,VLO log retn,WMB log retn,XOM log retn
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-15,0.022263,0.025396,-0.008728,-0.033953,-0.012635,0.007007,-0.006872,-0.028769,NaN,0.024402,...,0.032772,0.066663,0.029297,-0.021720,0.016702,-0.002283,0.112164,-0.003140,0.047666,-0.000636
2020-06-16,0.034209,0.029529,0.039798,-0.004650,0.030230,0.031312,0.044760,-0.021778,NaN,0.037725,...,0.020921,0.052391,0.063061,0.018868,0.011834,0.024912,0.038853,0.039620,0.012474,0.022237
2020-06-17,-0.047795,-0.044938,-0.024460,-0.021990,-0.026838,-0.051689,-0.039602,-0.004271,NaN,-0.070896,...,-0.039863,-0.059828,-0.063586,-0.055510,0.014221,-0.025632,-0.072190,-0.059794,-0.019823,-0.033115
2020-06-18,0.023480,0.000621,0.009721,-0.006373,0.004360,0.025021,0.008538,-0.024550,NaN,0.000859,...,0.035454,0.004964,0.032552,0.045072,0.017496,-0.016484,0.022348,0.036396,0.024462,0.006200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-02,0.018638,0.007261,0.015924,0.031183,0.008305,0.028666,0.025193,0.027374,0.026931,0.022342,...,-0.012458,0.020448,0.016055,0.007025,0.008736,-0.002061,0.027603,-0.005209,0.012482,0.007305
2025-06-03,0.052821,0.005878,0.013629,0.014252,0.012329,0.027555,0.013205,-0.000176,-0.009094,0.033519,...,0.020081,0.008808,0.029718,0.014680,0.022833,0.014514,0.015767,0.020444,-0.002615,0.007252
2025-06-04,-0.019343,-0.006682,-0.022548,-0.021857,-0.015745,-0.014158,-0.018338,-0.020689,-0.017234,-0.020808,...,-0.031484,-0.033222,-0.018440,-0.030556,-0.017446,-0.021393,-0.026106,-0.034257,-0.016333,-0.014556


In [47]:
# Divide data into training set and test set
train,test=train_test_split(holdingsreturn,test_size=0.2,random_state=100)

# Divide training set into training and validation set
train,validation=train_test_split(train,test_size=0.25,random_state=100)

In [48]:
# Let's take a look at the size of the data for the NN modeling
print("train:",train.shape)
print("validation:",validation.shape)
print("test:",test.shape)

train: (751, 24)
validation: (251, 24)
test: (251, 24)
